# Sufficiency and Efficiency in Deep RL

_W. Evan Durno, 2023_

**TODO:** the whole document contains a mixture of multivariate and univariate representations of $\theta$. Switch-over to multivariate.  

This work produces miniaturization and optimal efficiency results for reinforcement learning (RL). 
Miniaturization is achieved by modernizing the definition of sufficient statistics to accommodate the needs of deep learning, 
and thus produces infinite storage under the right circumstances. 
The optimal efficiency result is achieved by modernizing the Cramer-Rao Lower Bound (CRLB) away from classical statistics 
and toward the needs of RL, where sampling distributions can change. 

The key observation is that RL agents' change their sampling distributions as they learn. 
This violates the simple random sampling assumptions of classical statistics, 
thereby making old results less applicable. 
To account for this change, this work introduces the concept of a sufficient statistic regularizer (SSR), 
an approximately correct sufficient statistic applicable to deep learning. 
Further, we adjust the CRLB to account for smoothly deforming sampling distributions, 
and discover SSRs are used to yield optimally efficient data utilization in RL. 

## Introduction 

TODO: Describe the problem precisely 

TODO: Cover similar research. 
Mention Yann LeCun's work on memory. 
Motivate targetting RL with "RL is enough". 

## SSRs 

**Sufficient statistics as regularizers** 

Data reduction is the purpose of sufficient statistics. 
This becomes particularly apparent when studying their behavior under maximum likelihood estimation. 
The Fisher-Neyman defintion of a sufficient statistic $T(x)$ is that there 
exists $g$ and $h$ for density $f$ such that $f(x; \theta) = h(x) g(T(x) ; \theta)$. 
Notice that under maximum likelihood estimation, the $h$ term becomes irrelevant, leaving only $T(x)$, 
thereby providing an opportunity to reduce dimensionality of all data stored. 

$ \hat \theta = \arg \max_\theta f(x; \theta) = \arg \max_\theta \log f(x; \theta) $
$ = \arg \max_\theta \log(h(x)) + \log(g(x; T(x))) = \arg \max_\theta \log(g(x; T(x)))$ 

For example, dataset $x$ may require $O(n)$ space to store, but for $\theta \in \mathbb{R}^p$, 
it's common that $T$ only require $O(p)$ storage space. 
So, for a model with fixed parameter dimension, 
it is reasonably possible to have theoretically infinite data storage in a finite space. 
Of course, the amount of information truly stored will be practically bounded. 

For the purposes of this work, it is convenient to view sufficient statistics as regularizers. 
In situations where new data is added to old, like RL, this is particularly relevant. 
For example, with old data $X_A$ packed into $T_A = T(X_A)$ supplanting $\hat \theta_A$, 
we may add data $X_B$ ultimately producing $T_B = T(X_B)$ sufficiently estimating $\theta_B$. 

$\hat \theta_B = \arg\max_\theta f_X(X; \theta) = \arg \max_\theta \log f_X(X_A;\theta) + \log f_X(X_B;\theta) $
$ = \arg\max_\theta \log f_X(X_A;\theta) + \log g(T(X_B; \theta))$

Here, $X$ is the vector containing both $X_A$ and $X_B$, concatenated. 

If we then insert a scalar multiple $\lambda$, we recover the familiar regularizer form. 

$\hat \theta_B = \arg\max_\theta \log f_X(X_A;\theta) + \lambda \log g(T(X_B; \theta)) $

Let this form of regularizer be the SSR. 
Like other regularizers, $\lambda \log g(T(X_B); \theta)$ causes $\theta$ to 
stay near some point $\theta_A$. 
For example and without loss of generality (WLOG), $\lambda \| \theta \|^2$ centers $\theta$ on zero. 

**Universal SSRs via approximation** 

Data reduction via sufficient statistics is like compression. 
A breadth of theory has been developed for losses compression. 
Here, we'll highlight the benefits of lossy compression 
by relaxing our sufficient statistic definition to accommodate an approximation. 
Define $X_n \approx_{a.s.} Y$ to mean $\lim_{n \to \infty} X_n = Y \; a.s.$, 
and $X_n \approx_{\mathbb{P}} Y$ to mean $\lim_{n \to \infty} X_n = Y$ in $\mathbb{P}$.
Then, instead of defining $T$ sufficient if $f_X(x; \theta) = h(x) g(T(x); \theta)$, 
we'll accept $f_X(x; \theta) \approx_{\mathbb{P}} h(x) g(T(x); \theta)$ for large sample size $n$. 

With this relaxation, we are free to create approximate universal sufficient statistics 
only requiring sufficient regularity assumptions apply that the central limit theorem (CLT) 
apply to make the log likelihood normally distributed. 

First, recognize that the following Taylor expansion is accurate for $\theta$ near $\theta_A$. 

$n^{-1} \log f_X(X;\theta) \approx n^{-1}\log f_X(X; \theta_A) $
$ + (\theta - \theta_A) n^{-1}\frac{\partial}{\partial \theta} \log f_X(X; \theta_A)$
$ + 2^{-1}(\theta - \theta_A)^2 n^{-1} \frac{\partial^2}{\partial \theta^2} \log f_X(X; \theta_A) $  

$\approx_{a.s.} \mathbb{E} \log f_X(X_1; \theta_A) + (\theta - \theta_A) \mathbb{E} \frac{\partial}{\partial \theta} \log f_X(X_1; \theta_A)$ 
$ + 2^{-1} (\theta - \theta_A)^2 \mathbb{E} \frac{\partial^2}{\partial \theta^2} \log f_X(X_1;\theta_A) $ (apply the strong law of large numbers (SLLN))

$ = \mathbb{E} \log f_X(X_1; \theta_A) + 0 - 2^{-1} (\theta - \theta_A)^2 \mathcal{I}_{\theta_A} $

Here, we utilize further approximations:
- $\hat{\mathcal{I}}(X_A) = \hat{\mathcal{I}} = n^{-1}\sum_{i=1}^n G_i G_i^T \approx_{a.s.} \mathcal{I}$ 
where $G_i = \nabla_\theta \log f_X(X_i; \theta_A) $ 
- $\hat \theta_A(X_A) = \hat \theta_A \arg \max_\theta f_X(X_A;\theta) \approx_{\mathbb{P}} \theta_A$

Then we realize the following. 

$\mathbb{E} \log f_X(X_1; \theta_A) - 2^{-1} (\theta - \theta_A)^2 \mathcal{I}_{\theta_A}$

$\approx_{\mathbb{P}} \mathbb{E} \log f_X(X_1; \theta_A) - n_A 2^{-1} (\theta - \hat \theta_A(X_A))^2 \hat{\mathcal{I}}(X_A)$

Here, $n_A$ is the sample size of $X_A$. 

This approximation is not-yet useful, still depending on $\theta_A$. 
So, we must apply it in maximum likelihood estimation to drop the $\mathbb{E} \log f_X(X_1; \theta_A)$ term. 
Here, we add new data $X_B$ to the sample, while approximately retaining all information of $X_A$ 
in $T(X_A) = \left(\hat \theta_A(X_A), \hat{\mathcal{I}}(X_A) \right)$.

$\hat \theta_B = \arg\max_\theta f_X(X; \theta) = \arg\max_\theta \log f_X(X_B; \theta) + \log f_X(X_A;\theta) $

$ \approx_{\mathbb{P}} \arg\max_\theta \log f_X(X_B; \theta) $
$+ \mathbb{E} \log f_X(X_1; \theta_A) - n_A 2^{-1} (\theta - \hat \theta_A(X_A))^2 \hat{\mathcal{I}}(X_A) $

$ = \arg\max_\theta \log f_X(X_B; \theta) - n_A 2^{-1} (\theta - \hat \theta_A(X_A))^2 \hat{\mathcal{I}}(X_A) $

We thus identify the universal SSR $2^{-1} (\theta - \hat \theta_A(X_A))^2 \hat{\mathcal{I}}(X_A)$ 
with its natural regularization parameter value $\lambda = n_A$. 

**Miniaturization** 

Storing $\hat \theta(X_A)$ takes $O(p)$ space and $\hat{\mathcal{I}}(X_A)$ takes $O(p^2)$. 
So, while technically finite relative to $O(n)$, the sheer size of modern deep learning models 
makes any $O(p^2)$ storage requirement infeasible. 
So, this work will leverage a series of approximations that keep practically effective approximations 
to $\mathcal{I}$ in $O(p)$. 
- The simplest approximation is only storing diagonal terms, and zeroing all others. 
- A better (but usually unnecessary) approach is leveraging a Krylov estimate to 
approximate the major eigenvectors of $\mathcal{I}$. 
These are calculated via a modified version of the Lanczos algorithm. 

For experimental validation showing these approximations are effective, see Appendix B.

## Optimal MLE efficiency under deforming distributions 

As an RL agent learns from its environment, it tries new things and produces new kinds of data. 
Accepting that RL doesn't sample from identical distributions invites us to find an optimal transition paradigm. 
Interpreting _optimality_ as _efficiency_, this means working with the Cramer-Rao lower bound (CRB). 
Particularly, since maximum likelihood estimates (MLEs) from different distributions appear biased to another, 
we'll work with the mean squared error (MSE) form of the CRB. 
The strategy is simple: calculate the MSE under biased MLE deformations, and minimize it with respect to $\lambda$. 

To recover a optimal $\lambda$, we use the following definitions. 
- $X_A$ is the vector (or matrix) of observations having density $f_X(x; \theta_A)$. 
- $X_B$ similarly has density $f_X(x; \theta_B)$. 
- $\hat \theta_A = \arg\max_\theta f_X(X_A; \theta)$ is a simple MLE. 
- $\hat \theta_B = \arg\max_\theta f_X(X_B; \theta)$
- $\hat \theta_{AB} = \arg\max_\theta \log f_X(X_A ; \theta) + \log f_X(X_B; \theta) $ 
$ = \arg\max_\theta \sum_{i \in A} \log f_X(X_i ; \theta) + \sum_{i \in B} \log f_X(X_i; \theta) $ 
- $n_A = \#A, n_B = \#B, n = n_A + n_B$
- $p = \lim_{n \to \infty} n_B/n$. WLOG, we assume this series converges so we may have $p \approx n_B/n$. 

With these definitions in-place, we can realize this $\theta_{AB}$ form which can optimize MSE in $p$, determining optimal $\lambda$. 

$\hat \theta_{AB} = \arg\max_\theta \log f_X(X_A ; \theta) + \log f_X(X_B; \theta) $

$ \approx_{\mathbb{P}} \arg\max_\theta \log f_X(X_B; \theta) - n_A 2^{-1} (\theta - \hat \theta_A)^T \hat{\mathcal{I}} (\theta - \hat \theta_A)$

$ \approx n^{-1}\arg\max_\theta \log f_X(X_B; \theta) - (1-p) 2^{-1} (\theta - \hat \theta_A)^T \hat{\mathcal{I}} (\theta - \hat \theta_A)$

**CRB for mixed data** 

The CRB was designed with independent and identically distributed (iid) observations in-mind. 
Given our core observation that RL deforms distributions during sampling, 
we'll apply some regularity assumptions on our distributions to keep the CRB relevant. 
Assume that: 
- $f_X(x; \theta)$ is smooth in $\theta$ near $\theta_B$. 
- $\theta_A$ is sufficiently near to $\theta_B$ that $\log f_X(x; \theta_A) \approx \log f_X(x; \theta_B) + (\theta_A - \theta_B)^T \nabla_\theta \log f_X(x; \theta_B) $
$+ 2^{-1} (\theta_A - \theta_B)^T \left( \nabla^2_\theta \log f_X(x; \theta_B) \right) (\theta_A - \theta_B) $ holds.

This implies both $\theta_A$ and $\theta_B$ have approximately the same Hessians, $-\mathcal{I}$. 
Leveraging equivalent Hessians, the CRB can be applied to minimize the MSE. 

For bias term $b = \mathbb{E} \hat \theta_{AB} - \theta_B$ and $b' = \nabla_{\theta_B}b$, the CRB clearly states this effect of a biased estimate.

$\mathbb{E}(\hat \theta_{AB} - \theta_B) \geq (1 + b')^2 \mathcal{I}^{-1} + b^2$

Recognizing $\mathbb{E}\hat \theta_{AB} \approx p \theta_B + (1-p) \theta_A$, we recover this inequality. 

$\mathbb{E}(\hat \theta_{AB} - \theta_B) \gtrsim (1 - 1 + p)^2/(p\mathcal{I} + (1-p) \mathcal{I}) + (1-p)^2(\theta_A - \theta_B)^2 $
$ = p^2 \mathcal{I}^{-1} + (1-p)^2 (\theta_A - \theta_B)^2$

Differentiating by $p$ and setting to zero, we recover an optimal approximate value for $\lambda$. 

$\frac{n_A}{n_B} \approx \frac{p}{1-p} = (\theta_B - \theta_A)^2 \mathcal{I} $

Using the natural value for $\lambda = n_A$, we recover $\hat \lambda = n_B (\theta_A - \theta_B)^2 \mathcal{I} $.

Unfortunately, $\hat \lambda$ is used to estimate $\theta_B$, so we do not yet know $(\theta_A - \theta_B)^2$. 
However, it's fair that to assume that a sufficiently complex agent will eventually understand an entire game, 
resulting in true $\theta$ converging. In this case, the standard CRB applies, and eventually $(\theta_A - \theta_B)^2 \sim N(0, \mathcal{I}^{-1}/n_B)$, resulting in $\hat \lambda = 1$. 

Under this paradigm, $n_A$ is still useful when arbitrarily large. 
However, setting it to $n_A = \hat \lambda$ virtually up-samples $X_A$ or $X_B$. 

**Experimental results: Cart Pole**

This initial experiment demonstrates the effectiveness of SSRs, 
and studies the behavior of $\| \theta_A - \theta_B \|^2_F$ near the solution point. 
The experiment is kept purposefully simple to isolate experimental effects. 
We apply a [small deep net](https://github.com/wdurno/notebooks/commit/8e20a0ec7a5c2376b954d2a04a930064e8e77f69#diff-9067e1aea905e3792706d011ba5a0007ba0147bbc9f8b02e76a0b9b07acbd13d) 
to [Cart Pole with a 4-dimensional state space](https://www.gymlibrary.dev/environments/classic_control/cart_pole/). 

Experimental conditions are 
1. SSRs are applied with $\lambda = 1$. The observation cache is entirely cleared after each memorization event.
2. SSRs are applied with approximately optimal $\lambda = \hat \lambda = (\theta_A - \theta_B)^T \hat{\mathcal{I}} (\theta_A - \theta_B)$. Since $\theta_A$ is never known, we use the $\theta_A - \theta_B$ difference from the prior memorization event. The observation cache is entirely cleared after each memorization event.
3. SSRs are applied with $\lambda = \hat \lambda / 10$. The observation cache is entirely cleared after each memorization event.
4. SSRs are applied with $\lambda = \hat \lambda * 10$. The observation cache is entirely cleared after each memorization event. 
5. Control: No SSRs are applied, but the observation cache is cleared at moments coinciding with each memorization event. 
6. Control: No SSRs are applied, and observations caches are entirely retained. 

**TODO:** clean-up this graphic: axes, title, number conditions 

In the below graphic, the x-axis tracks the game iteration, 
and the y-axis tracks average game score over 1000 agents. 
Agents restart their games when they lose and during memorization events. 
Three memorization events can clearly be seen in the graph.

![cart pole](./data/df-experiment-22.png)


This graphic clearly supports the optimality of the SSR method in RL applications. 
Lift is clearly more-efficient, as the method uses data most-efficiently. 
Better yet, the method enjoys the miniaturization benefits of storing any amount of data in an $O(p)$ space. 

It can be seen that the lagged estimate of $\hat \lambda$ is decently accurate, 
but so is taking $\hat \lambda = 1$ even during early gameplay. 

**Experimental result: robotics**

This work's sufficiency and efficiency results are asymptotic results, 
so are theoretically valid for a large breadth of models. 
So, the sufficiency and efficiency should be demonstrated in another game beyond Cart Pole.
Further, sufficiency and efficiency are particularly useful toward robotics, 
this new game is played by a real-world robot. 

The robot is a PiCar V \[1\] wheeled robot with articulated camera, 
capable of driving forward & backward, turning left & right, 
and looking up, forward, left & right. 
Video processing and servo articulation is processed by an on-board Raspberry Pi 4B.

![robot](./data/robot.jpg)

This PiCar is customized to minimize on-board processing, 
instead running [this](https://github.com/wdurno/picar-v-rl-env/blob/1d5f5e15b390ef7cceae3fbe8875f71fbc76995f/run_api.py) server, offloading deep net processing to 
[this](https://github.com/wdurno/notebooks/blob/814f7cfed779fc4990f770b20a5ad924fbd7c693/regularizers-as-memory/car.py)
GPU-accelerated client. 

Similar to the out-of-the-box [software](https://github.com/sunfounder/SunFounder_PiCar-V/blob/master/ball_track/ball_tracker.py), 
the RL game is to chase a red ball and have the agent get close to it, 
confirming the presence visually. 
Experiments were conducted in the same household office, 
under near-consistent lighting conditions. 

Three experiments were run.

1. (Control) standard Q-Learning, retaining all data in the memory buffer 
2. (Experimental 1) Using memorization, clearing the memory buffer after each memorization, 
using $\hat \lambda_t = (\hat \theta_{t} - \hat \theta_{t-1})^T \hat{\mathcal{I}}(\hat \theta_{t} - \hat \theta_{t-1}) $, where $\hat{\mathcal{I}}$ is estimated with a Krylov rank of 10 (see Appendix A).
Memorization occurs 40 times.
3. (Experimental 2) Same as experimental 1, but with $\hat \lambda = 1$ for every iteration.

Data were all sampled by an agent following experimental condition 2.
All three experimental conditions used the same datasets, 
with the agent being exposed to data in the same order it was sampled. 
Thus, this was an offline RL experiment. 
Since optimal offline RL metrics are unclear \[2\], 
we simply use expected discounted reward per batch at teach time step. 

**TODO**: This graphic is _terrible_. Also, not pictured: $\hat \lambda = 1$ is lowest performer.
![robot metrics](./data/df-experiment-24.png)

As can be seen, this new game has exceptional results when 
SSRs are applied and $\hat \lambda$ is projected. 
This illustrates the potential for universal effectiveness of SSR efficiency. 
Further, the failure of the $\hat \lambda = 1$ case shows that complex games 
require accurate $\hat \lambda$ projection. 

## Discussion 

TODO: SSRs provide a universal miniaturization paradigm 

TODO: RL lift should be expected by accounting for deforming distributions 

TODO: $\hat \lambda$ projection needs further work. 

TODO: The other half of CRLB $\mathcal{I}^{-1}/n$ can be explored. 
I've optimized $n$, but there's a rich structure to $\mathcal{I}$. 

TODO: Continuous sampling and Ito integrals? 

TODO: Robotics ought to be tested again with a real budget 

TODO: alternative methods to estimating $\hat{\mathcal{I}}$ and $\| \hat \theta_B - \hat \theta_A \| $. 

## References 

\[1\] https://www.sunfounder.com/products/smart-video-car

\[2\] Romain Deffayet, Thibaut Thonet, Jean-Michel Renders, & Maarten de Rijke (2022) "Offline Evaluation for Reinforcement Learning-based Recommendation: A Critical Issue and Some Alternatives", ACM SIGIR Forum, Vol. 56 No. 2. 

## Appendix A: modified Lanczos algorithm 

TODO: Describe the $LL^T + \Lambda$ estimate, and motivate it with Shun Ichi Amari's work on eigenvalue distributions. 

TODO: my limited-memory Lanczos algorithm 

## Appendix B: Effectiveness of $O(p)$ Hessian approximations 

TODO: Show results 